# Imports
De benodigde libraries importeren.

In [19]:
from mesa import Agent, Model
import random
from mesa.space import SingleGrid
from mesa.time import BaseScheduler
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner

# De CarAgent class
In deze class wordt de auto (agent) gemaakt, elke auto heeft twee waardes: de snelheid en zijn posiitie op de weg. In de functie 'optrekken' krijgt de auto de mogelijkheid zoals de naam al verklapt: op te trekken. Dit doet de auto als hij niet de maximum snelheid rijdt, in dit geval als de snelheid onder de 5 is dus. 'remmen' De auto remt als de cel waar de auto heen wilt gaan al bezet is door een andere auto, als dit het geval is wordt de snelheid meet één verminderd, daarna wordt de functie nog een keer aangeroepen om te kijken of het dan nog niet past wordt de snelheid weer met één verminderd totdat het wel past. In de functie 'move' krijgt de auto de mogelijkheid om te verplaatsen, de auto zal verplaatsen het aantal cellen vooruitgaan wat gelijk staat aan de snelheid van de auto, dus als de auto nu op cel 42 staat met een snelheid van 4 zal de auto naar cel 46 gaan (als cel 46 niet bezet is dan, als dat wel het geval is gaat de functie 'remmen' in werking). 'role_of_randomization' Hier  heeft de auto een 50% (dit percentage kan veranderd worden) kans dat de snelheid afneemt met één.

In [20]:
class CarAgent(Agent):

    max_snelheid = 5

    # Elke auto begint met een random snelheid, van 3 T/M 5.
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.snelheid = random.randint(3, 5)
        self.pos = pos


    def optrekken(self):
        if self.snelheid < CarAgent.max_snelheid:
            self.snelheid += 1

        return self.snelheid

    def remmen(self):
        if self.model.grid.is_cell_empty(self.model.grid.torus_adj((self.pos[0]+self.snelheid, self.pos[1]))):
            self.snelheid == self.snelheid
        else: 
            if self.snelheid > 0:
                self.snelheid -= 1
                self.remmen() 

        return self.snelheid

    def move(self):
        if self.model.grid.is_cell_empty(self.model.grid.torus_adj((self.pos[0]+self.snelheid, self.pos[1]))):
            self.model.grid.move_agent(self,self.model.grid.torus_adj((self.pos[0]+self.snelheid, self.pos[1])))

    def role_of_randomization(self):
        if self.snelheid > 1:
            if random.randint(0,1) == 1:
                self.snelheid -= 1
            else:
                self.snelheid = self.snelheid

        return self.snelheid
    
    def step(self):
        self.optrekken()
        self.remmen()
        self.move()
        self.role_of_randomization()
        print ("Ik ben auto: " + str(self.unique_id) + " Snelheid:" + str(self.snelheid))
        print(self.pos)

# De CarModel class
In deze class gebeuren een paar dingen, de auto's worden gemaakt en krijgen een random positie op de weg en er wordt data verzameld. Ook worden hier de dementies van de weg gemaakt, de breedte (oftewel de lengte van de weg). Voor dit onderzoek was het data collecten een van de belangrijke dingen, alleen is er één probleem, de posities van de auto kunnen niet goed opgeslagen worden. Daarom heb ik ervoor gezorgd dat de informatie ook geprint wordt.

Een voorbeeld:
Ik ben auto: (72, 5) Snelheid:4
(4, 0)
Dit is een mogelijke output, de naam hier is (72,5), de snelheid is 4 en de positie is (4,0) dit betekent dat hij op cel 4 staat van de weg. Het tweede getal is altijd nul in dit geval omdat er maar één weg is.

In [21]:
class CarModel(Model):
    
    def __init(self, height, width, car_amount):
        super().__init__()
        self.height = height
        self.width = width
        self.car_amount = car_amount
        
        self.schedule = BaseScheduler(self)
        self.datacollector = DataCollecter(
            agent_reporters = {"Snelheid": "snelheid"})
        self.grid = SingleGrid(height, width, torus=True)
        
        self.running = True
        
    def maak_agent(self):
        for i in range(self.car_amount):
            while True:
                try:
                    r =  random.random()
                    agent = CarAgent((int(r*100), 5), self)
                    self.grid.position_agent(agent, int(r*100))
                    self.schedule.add(agent)
                    break
                except Exception as e:
                    continue

        self.maak_agent()
                    
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

### Het runnen van de simulatie zonder visualisatie
Hier wordt de class CarModel aangeroepen, in dit geval met de waardes: (100,1,45) Dit betekent dat de weg 100 cellen lang is, één hoog en er 45 auto's op rijden. We zullen de simulatie met meerde hoeveelheden auto's laten draaien, de andere twee waardes zullen hetzelfde blijven. Elke simulatie zal 1000 steps hebben, een step kan gezien worden als een beurt, in elke step komt elke auto één keer aan de beurt.

In [22]:
model = CarModel(100, 1, 10) 
for i in range(1000):
    model.test()

AttributeError: 'CarModel' object has no attribute 'test'

In [ ]:
agent_snelheden = model.datacollecter.get_agent_vars_dataframe()
agent_snelheden.plot()

In [ ]:
agent_snelheden.mean()

In [ ]:
end_snelheden = agent_snelheden.xs(1000, level="Step")["Snelheid"]
end_snelheden.hist(bins=range(agent_snelheden.Snelheid.max()+1))

In [8]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter

def car_draw(agent):

    if agent.snelheid > 3:
        color = 'green'
    if 1 < agent.snelheid < 4:
        color = ' orange'
    if agent.snelheid < 2:
        color = 'red'
    return {"Shape": "rect", "w": 0.5,"h": 40, "Filled": "true", "Layer": 0, "Color": color, "stroke_color": "#00FF00" }

canvas_element = CanvasGrid(car_draw, 100, 1, 500, 500)

model_params = {
    "height": 100,
    "width": 1,
    "car_amount": UserSettableParameter("slider", "Number of Cars", 10, 1, 100 , 1)
}

server = ModularServer(CarModel,
                       [canvas_element],
                       "Nagel-Schreckenberg", model_params)
server.port = 8529
server.launch()

Interface starting at http://127.0.0.1:8529


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
